In [1]:
from IPython.display import display, Image, clear_output, HTML
from jupyter_ui_poll import ui_events
from bs4 import BeautifulSoup
import time
import numpy as np
import requests
import json
import ipywidgets as widgets
import pandas as pd

def next():
    next= input("Press Enter to continue.")
    return next
    
def buttons():

    button_messages= ["Yes", "No"]
    btn1= widgets.Button(description= button_messages[0], style= {"button_color": "green"})
    btn2= widgets.Button(description= button_messages[1], style= {"button_color": "red"})
    btn1.on_click(register_event) 
    btn2.on_click(register_event)
    end_panel= widgets.HBox([btn1, btn2])
    return end_panel, button_messages
    
event_info = {'type': '',
              'description': '',
              'time': -1}

def wait_for_event(interval=0.001, max_rate=20, allow_interupt=True):

    start_wait = time.time()
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1
    n_proc = int(max_rate*interval)+1

    with ui_events() as ui_poll:
        keep_looping = True
    
        while keep_looping==True:
            ui_poll(n_proc)
            
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
            
            time.sleep(interval)

    return event_info
    
def register_event(btn):

    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return
    
def send_to_google_form(data_dict, form_url):

    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'
    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    loaded_all = True

    for item in result:
    
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]

    post_result = requests.post(post_form_url, data=form_dict)

    return post_result.ok
    
def messages(correct_answers, level_number, final_score, average_correct_time, average_time, name, style, user_id, gender, sleep, ia_json): 
    data_dict= {
    "user_id": user_id,
    "correct_answers": correct_answers,
    "final_score": final_score,
    "average_correct_time": average_correct_time,
    "average_time": average_time,
    "gender": gender,
    "sleep": sleep,
    "ia_json": ia_json
    }
    form_url= "https://docs.google.com/forms/d/e/1FAIpQLSfXDFoeFVyEsiVTw2wqDfnQMhSFxVLy79u4hBBGUSV-aFdsaA/viewform?usp=sf_link"
    start_messages= [
        "Welcome to the Maths Ability Test!",
        "What is your name? (NOT YOUR ID)",
        "Please enter your anonymised ID.",
        "What is your gender? Please enter male, female, non-binary or prefer not to say.",
        "How many hours of sleep did you have last night?",
        "You will be shown some arithmetic questions. However, each question will be split into different parts and each part will be shown seperately for a short amount of time.",
        "Enter the answer when prompted, as fast as possible. You will be scored according to your accuracy and speed.",
        "Please DO NOT type numbers unprompted, as this would exit the code.",
        "There are 3 levels, and 5 questions in each level.",
        "3",
        "2",
        "1"
    ]   
    end_messages= [
        f"Well done, {name}!",
        f"Not bad, {name}.",
        "You should carry a calculator at all times.",
        "You did not answer any questions correctly.",
        f"You answered {correct_answers}/ 15 questions correctly, {correct_answers- 11} more than the average, 11!",
        f"You answered {correct_answers}/ 15 questions correctly, the same as the average person!",
        f"You answered {correct_answers}/ 15 questions correctly, {11- correct_answers} fewer than the average, 11.",
        f"You scored: {final_score:.1f}/ 100,{49.3- final_score: .1f} lower than the average score, 49.3.",
        f"You scored: {final_score:.1f}/ 100,{final_score- 49.3: .1f} higher than the average score, 49.3!",
        f"You scored: {final_score:.1f}/ 100, exactly the same as the average person!",
        f"You spent on average {average_correct_time:.2f}s for each question you answered correctly,{average_correct_time- 4.01: .2f}s slower than the average, 4.01s.",
        f"You spent on average {average_correct_time:.2f}s for each question you answered correctly,{4.01- average_correct_time: .2f}s faster than the average, 4.01s!",
        f"You spent on average {average_correct_time:.2f}s for each question you answered correctly, exactly the same as the average person!",
        "We wish to record your response data to an anonymised public data repository.",
        "Your data will be used for educational teaching purposes, specifically for practicing and teaching data analysis and visualisation.",
        "Please click Yes if you consent to the upload.",
        "Thanks- your data will be uploaded.",
        "Please contact a.fedorec@ucl.ac.uk if you have any questions or concerns regarding the stored results.",
        "No problem, we hope you enjoyed the test.",
        "Thank you for your time!"
    ]
    end_panel, button_messages= buttons()

    if level_number== 0:

        for message in start_messages: 
            print(message)
    
            if message== start_messages[1]:
                name= input("")
                clear_output()
                print(f"Hi {name}!")
                time.sleep(2)

            elif message== start_messages[2]:
                id_instructions= """
                Enter your anonymised ID
                To generate an anonymous 4-letter unique user identifier please enter:
                -two letters based on the initials(first and last name) of a childhood friend
                -two letters based on the initials(first and last name) of a favourite actor/ actress
                e.g. if your friend was called Charlie Brown and film star was Tom Cruise
                then your unique identifier would be CBTC
                """
                print(id_instructions)
                user_id= input(">")
                clear_output()
                print("User entered id:", user_id)
                next()
                clear_output()

            elif message== start_messages[3]:
                
                while gender not in ["male", "female", "non-binary", "prefer not to say"]:
                    gender= input("Enter gender:")

                if gender== "male":
                    gender= "m"

                elif gender== "female":
                    gender= "f"

                clear_output()

            elif message== start_messages[4]:
                
                while sleep== "" or type(sleep)!= float:
                    sleep= input("Enter number:")
                    
                    try:
                        sleep= float(sleep)
                        
                    except ValueError:
                        print("Please try again.")
                        
                clear_output()
                        
            elif message== start_messages[5]:
                time.sleep(4)
        
            elif message== start_messages[8]:
                time.sleep(2)
                input("Press Enter when you're ready to begin!")
                clear_output()
                print(f"Good luck, {name}!")
                time.sleep(2)
        
            elif message== "3" or "2" or "1":
                time.sleep(1)
        
            else:
                time.sleep(2)

    elif level_number== 3:

        if correct_answers>= 1:
    
            if correct_answers>= 12:
                print(end_messages[0])
                time.sleep(2)
                print(end_messages[4])
    
            elif correct_answers>= 8:
                print(end_messages[1])
                time.sleep(2)
                
                if correct_answers== 11:
                    print(end_messages[5])
                    
                elif correct_answers<= 10:
                    print(end_messages[6])
    
            else:
                print(end_messages[2])
                time.sleep(2)
                print(end_messages[6])

            time.sleep(2)
            
            if float(f"{final_score: .1f}")< 4.93:
                print(end_messages[7])
                
            elif float(f"{final_score: .1f}")> 50.9:
                print(end_messages[8])
                
            else:
                print(end_messages[9])

            time.sleep(2)
            
            if float(f"{average_correct_time: .2f}")> 3.92:
                print(end_messages[10])
                
            elif float(f"{average_correct_time: .2f}")< 3.92:
                print(end_messages[11])
                
            else:
                print(end_messages[12])
                
        else:
            print(end_messages[3])

        next()
        clear_output()

        for message in end_messages[13:16]:
            print(message)
            time.sleep(2)

        display(end_panel)
        event_info= wait_for_event()
        clear_output()
    
        if event_info['description']== button_messages[0]:
            print(end_messages[16], end_messages[17])
            send_to_google_form(data_dict, form_url)

        else:           
            print(end_messages[18])

        time.sleep(2)
        print(end_messages[-1])

    return name, user_id, gender, sleep
        
def maths_ability_test():

    level_1= np.array([[9, "x9", -4],
                       [4, "x2", "x5"],
                       [8, "+9", -4],
                       [1, "x8", "+9"],
                       [4, -2, -3]])
    level_2= np.array([[12, -3, "x12"],
                       [10, "+5", "x13"],
                       [5, "x6", "x11"],
                       [7, "+13", "x12"],
                       [8, "+1", "x15"]])
    level_3= np.array([[4, "x14", "÷8"],
                       [18, "x18", "÷4"],
                       [132, "÷11", -20],
                       [196, "÷7", "+20"],
                       [144, "+18", "÷9"]])
    all_levels= [level_1, level_2, level_3]
    lvl1_answers= [77, 40, 13, 17, -1]
    lvl2_answers= [108, 195, 330, 240, 135]
    lvl3_answers= [7, 81, -8, 48, 18]
    all_answers= [lvl1_answers, lvl2_answers, lvl3_answers]
    row_number= 0
    level_number= 0
    correct_answers= 0
    final_score= 0
    average_correct_time= 0
    average_time= 0
    all_scores= []
    all_times= []
    all_correct_times= []
    name= ""
    style= f"font-size: 50px;"
    user_id= ""
    gender= ""
    sleep= ""
    individual_answers= []
    ia_json= ""
    name, user_id, gender, sleep= messages(correct_answers, level_number, final_score, average_correct_time, average_time, name, style, user_id, gender, sleep, ia_json) 

    for level in all_levels:
    
        for row in level:
            clear_output()
    
            for operation in row:
                number_out= HTML(f"<span style='{style}'>{operation}</span>")
                display(number_out)
                time.sleep(2)
                clear_output(wait= False)   
    
            start_time= time.time()
            ans= input("Enter answer:")

            try:
            
                if ans== "stop":
                    print("Maths Ability Test stopped.")
                    return
        
                elif int(ans)== all_answers[level_number][row_number]:
                    end_time= time.time()
                    time_taken= end_time- start_time
                    correct_answers= correct_answers+ 1 
                    all_correct_times.append(time_taken)
                    individual_answers.append(1)
                    print("Correct!")
                    print(f"You took {time_taken:.2f} seconds.")
                
                    for i in range(10):
            
                        if time_taken>= 8.2:
                            score= 5
                            break
            
                        elif time_taken<= 1:
                            score= 100
                            break
            
                        elif time_taken>= 0.8*i+ 1:
                            score= 100- 10*(i+ 1)
            
                        else:
                            break
        
                else:
                    end_time= time.time()
                    time_taken= end_time- start_time
                    print("Incorrect")
                    individual_answers.append(0)
                    score= 0
            
            except ValueError:
                end_time= time.time()
                time_taken= end_time- start_time
                print("Incorrect")
                individual_answers.append(0)
                score= 0
        
            next()
            row_number= row_number+ 1
            all_scores.append(score)
            all_times.append(time_taken)
            clear_output()
        
        level_number= level_number+ 1
        row_number= 0

        if level_number<= len(all_levels)- 1:
            print(f"Level {level_number+ 1}")
            time.sleep(2)
            next()
            
    clear_output()
    final_score= sum(all_scores)/ (len(all_scores))
    average_time= sum(all_times)/ (len(all_times))
    
    if (len(all_correct_times))== 0:
        average_correct_time= 0
    else:
        average_correct_time= sum(all_correct_times)/ (len(all_correct_times))
        
    ia_dataframe= pd.DataFrame(individual_answers, columns= ["correct_answer"])
    ia_json= ia_dataframe.to_json()
    end_panel, button_messages= buttons()
    messages(correct_answers, level_number, final_score, average_correct_time, average_time, name, style, user_id, gender, sleep, ia_json)

In [ ]:
maths_ability_test()

Welcome to the Maths Ability Test!
What is your name? (NOT YOUR ID)
